In [2]:
#install packages
!pip install osmnx

     |████████████████████████████████| 102kB 6.2MB/s 
     |████████████████████████████████| 1.0MB 13.1MB/s 
     |████████████████████████████████| 10.3MB 27.4MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 6.6MB 38.4MB/s 
     |████████████████████████████████| 1.0MB 42.7MB/s 
     |████████████████████████████████| 15.4MB 266kB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [9]:
!pip install google-cloud
!pip install google-cloud-pubsub

     |████████████████████████████████| 215kB 7.4MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 522kB 13.1MB/s 
     |████████████████████████████████| 645kB 16.1MB/s 
  Created wheel for grpc-google-iam-v1: filename=grpc_google_iam_v1-0.12.3-cp37-none-any.whl size=18516 sha256=1b9a539e38d09e44356a0d1e7b32d9c0d268be378a51863761d7876f8a5f91a2
  Stored in directory: /root/.cache/pip/wheels/de/3a/83/77a1e18e1a8757186df834b86ce6800120ac9c79cd8ca4091b
Successfully built grpc-google-iam-v1
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import networkx as nx
import osmnx as ox
import pandas as pd
import time
import random
import os
import datetime
import pytz
from google.cloud import pubsub_v1

In [2]:
# Defining the map boundaries
north, east, south, west = 43.971564, -79.002807, 43.342461, -79.895757
# Downloading the map as a graph object
G = ox.graph_from_bbox(north, south, east, west, network_type='drive')

In [3]:
# Define destination locations
s1 = (43.639492, -79.379882)  # harbour front
s2 = (43.641138, -79.388900)  # rodgers centre
s3 = (43.640113, -79.374857)  # ferry terminal
s4 = (43.651347, -79.382221)  # scotia bank
s5 = (43.653939, -79.378925)  # massey hall
s6 = (43.706773, -79.398314)  # yonge-eglington
s7 = (43.677557, -79.409451)  # casa loma
s8 = (43.636669, -79.421906)  # liberty village
s9 = (43.666982, -79.394483)  # rom
s10 = (43.651759, -79.397942)  # chinatown
s11 = (43.648570, -79.414358)  # trinity bellwoods park
s12 = (43.655312, -79.435155)  # dufferin mall
s13 = (43.648660, -79.464024)  # high park

major_points = [s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13]

In [4]:
# Define various user routes
users = 40
users_route = [None] * users

# number of points in routes
points_in_route = [3, 4]

# choose the random seed, so the random number is same
random.seed(1)

for i in range(users):
    points = random.sample(points_in_route, 1)[0]
    users_route[i] = random.sample(major_points, points)

,Unix_Time,User,Long,Lat


In [ ]:
cols = ['Unix_Time', 'User', 'Long', 'Lat']
master_df = pd.DataFrame(columns=cols)

In [5]:
user = 0
start_time = int(time.time())

1623250536


In [37]:
for u_r in users_route:
    user += 1
    time_l = []
    user_l = []
    long_l = []
    lat_l = []
    time_l = []
    user_current_time = start_time
    for i in range(0, len(u_r) - 1):
        #change from ox.get_nearest_node() to ox.distance.nearest_nodes() because the first function will be deprecated
        origin_node = ox.distance.nearest_nodes(G, u_r[i][0],u_r[i][1])
        destination_node = ox.distance.nearest_nodes(G, u_r[i + 1][0],u_r[i][1])
        # shortest_path() will return a list of the nodes constituting the shortest path between them
        route = nx.shortest_path(G, origin_node, destination_node, weight='length')
        
        #append each point in this route
        for index, j in enumerate(route):
            point = G.nodes[j]
            long_l.append(point['x'])
            lat_l.append(point['y'])
            user_l.append('user' + str(user))
            time_l.append(user_current_time)
            user_current_time += 10
            
    #append each route to the df
    df = pd.DataFrame(columns=cols)
    df['User'] = user_l
    df['Long'] = long_l
    df['Lat'] = lat_l
    df['Unix_Time'] = time_l
    master_df = master_df.append(df, ignore_index=True)

In [14]:
timestamps = master_df['Unix_Time'].unique().copy()
timestamps.sort()

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
project_id = "drvn-sb"
topic_name = "gis_users_for_cloud_function"
#service account key
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/gdrive/MyDrive/service_account/bq-looker-gis.json"

In [19]:
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_name)

In [21]:
len(master_df)

4037

In [22]:
i = 0
step = 100
while i <= master_df.shape[0] - 1:
    print(i)
    data_to_insert = master_df.loc[i: i + step]
    i = i + step + 1
    print("********************************DATA**************************************")
    print(data_to_insert)
    data = data_to_insert.to_json(orient='records')
    data = str(data).encode("utf-8")
    future = publisher.publish(topic_path, data)
    print("********************************RESULT**************************************")
    print(future.result())
    time.sleep(10)

0
********************************DATA**************************************
      Unix_Time   User       Long        Lat
0    1623250536  user1 -79.397732  43.651880
1    1623250546  user1 -79.397321  43.650865
2    1623250556  user1 -79.396467  43.648730
3    1623250566  user1 -79.396108  43.647843
4    1623250576  user1 -79.395907  43.647368
..          ...    ...        ...        ...
96   1623251056  user2 -79.436697  43.652720
97   1623251066  user2 -79.436985  43.653469
98   1623251076  user2 -79.437288  43.654253
99   1623251086  user2 -79.437610  43.655099
100  1623251096  user2 -79.437063  43.655213

[101 rows x 4 columns]
********************************RESULT**************************************
2514093636526304
101
********************************DATA**************************************
      Unix_Time   User       Long        Lat
101  1623251106  user2 -79.436436  43.655331
102  1623250536  user3 -79.464730  43.645631
103  1623250546  user3 -79.458311  43.644207
104  1

KeyboardInterrupt: ignored